# NSPP Generator

Generate observations of a non-stationary Poisson Process -- like arrivals to a customer-centered system.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

## Setting Things Up

In [ ]:
# defaults
dsname = 'x' # A, B, or C for the video, anything else, otherwise
arrival_rates = np.array([10.0, 60.0, 42.0, 20.0, 10.0, 5.0, 12.0, 15.0])
durations = np.zeros([len(arrival_rates)])
durations.fill(1.0)

#
# The experiment values for the video
#
if dsname == 'A':
    arrival_rates = np.array([35.0]) # A
    durations = np.array([8.0]) #A
elif dsname == 'B':
    arrival_rates = np.array([20, 50]) # B
    durations = np.array([4.0, 4.0]) #B
elif dsname == 'C':
    arrival_rates = np.array([20.0, 70.0, 40.0, 10.0]) # C
    durations = np.array([2.0, 2.0, 2.0, 2.0])  #C

if len(arrival_rates) != len(durations):
    print("Error -- lengths of arrs and durrs much match")
# probabilites of retention
prs = arrival_rates / np.max(arrival_rates)
# end times
ends = np.zeros(len(durations))
ends[0] = durations[0]
for j in range(1, len(durations)):
    ends[j] = ends[j-1]+durations[j]
# total time
total_time = np.sum(durations)
num_bins = int(total_time)
# max arrival rate
max_arr = np.max(arrival_rates)
l = 1/max_arr
print("     Num. Periods: {:}".format(len(arrival_rates)))
print("         Arrivals: {:}".format(arrival_rates))
print("        Durations: {:}".format(durations))
print("           Probs.: {:}".format(prs))
print("       Total time: {:} ({:} histogram bins)".format(total_time, num_bins))
print("             Lmax: {:.3f}".format(l))
print("Mean arrival rate: {:.2f}".format(np.mean(arrival_rates)))
print("         Mean IAT: {:.4f}".format(1/np.mean(arrival_rates)))

# The arrival rate process
fig = plt.figure(figsize=(8, 4))
ax = plt.axes()
p = ax.bar(np.cumsum(durations)-.5*durations[0],arrival_rates, durations)

In [ ]:
arrival_rates, durations

## Define the generate() function

In [ ]:
# Generate the arrival times (arrs) and interarrival times (iats)
# based on the max arrival rate (l), ends, and prs
def generate(l, ends, prs) :
    arrs = []
    last = len(ends)-1
    tnow = 0.0
    current = 0
    while True:
        iat = np.random.exponential(l)
        tnow += iat
        # done?
        if tnow > ends[last]:
            break
        # need to move to the next period?
        elif tnow > ends[current]:
            current += 1
        # thinning
        if np.random.random() < prs[current]:
            arrs.append(tnow)
    # Done with arrivals, now get the interarrival times
    iats = []
    iats.append(arrs[0])
    for j in range(1, len(arrs)):
        iats.append(arrs[j] - arrs[j-1])
    return (iats, arrs)

# trial run
# generate and plot one replication
iats, arrs = generate(l, ends, prs)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
v1 = ax[0].hist(iats)
v2 = ax[1].hist(arrs, bins=num_bins) 

In [ ]:
# What if it were actually a stationary Poisson process?
meaniat = np.mean(iats)
num = int(total_time / meaniat)
xiats = np.random.exponential(meaniat, num)
xarrs = np.cumsum(xiats)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
v1 = ax[0].hist(xiats)
v2 = ax[1].hist(xarrs, bins=num_bins) 
num, meaniat

## Run Multiple Replications

In [ ]:
reps = 10
alliats = []
allarrs = []
for j in range(reps):
    iats, arrs = generate(l,ends, prs)
    alliats.append(iats)
    allarrs.append(arrs)
    

In [ ]:
fig, ax = plt.subplots(nrows=reps, ncols=2, figsize=(10, 3*reps))
for j in range(reps):
    ax[j][0].hist(alliats[j])
    ax[j][1].hist(allarrs[j], bins=num_bins)

In [ ]:
# write output files
for j in range(len(allarrs)):
    fp = open('..\data\day{:}{:}.csv'.format(dsname, j+1), "w")
    fp.write("\n".join(["{:.3f}".format(t) for t in allarrs[j]]))
    fp.close
# not sure why I need this, but seems necessary for a final buffer write
del fp